In [4]:
import http.client, urllib.request, urllib.parse, urllib.error, base64

headers = {
    # Request headers
    'api_key': '7b346fab54ea430c912a9f22e7ee8053',
}

params = urllib.parse.urlencode({
    # Request parameters
    'Lat': '38.8978168',
    'Lon': '-77.0404246',
    'Radius': '700000',
})

try:
    conn = http.client.HTTPSConnection('api.wmata.com')
    conn.request("GET", "/Rail.svc/json/jStationEntrances?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

b'{"Entrances":[{"ID":"86","Name":"EAST ENTRANCE (EAST OF 17th & I STs)","StationCode1":"C03","StationCode2":"","Description":"Building entrance from the southeast corner of 17th St NW and I St NW(Escalator only).","Lat":38.901169,"Lon":-77.039164},{"ID":"85","Name":"WEST ELEVATOR ENTRANCE (WEST OF 17th & I STs)","StationCode1":"C03","StationCode2":"","Description":"Entrance elevator is located at the northwest corner of 18th St NW and I St NW.","Lat":38.901502,"Lon":-77.041888},{"ID":"88","Name":"WEST ENTRANCE 2(BUIDLING INTERIOR)","StationCode1":"C03","StationCode2":"","Description":"Station entrance escalator is located in the building (International Square) food court","Lat":38.90149,"Lon":-77.042054},{"ID":"87","Name":"WEST ENTRANCE (WEST OF 17th & I STs)","StationCode1":"C03","StationCode2":"","Description":"Station entrance escalator is located in the building courtyard, at the northwest corner of 18 St NW and I St NW.","Lat":38.901622,"Lon":-77.041965},{"ID":"7","Name":"SOUTH E

In [5]:
data

b'{"Entrances":[{"ID":"86","Name":"EAST ENTRANCE (EAST OF 17th & I STs)","StationCode1":"C03","StationCode2":"","Description":"Building entrance from the southeast corner of 17th St NW and I St NW(Escalator only).","Lat":38.901169,"Lon":-77.039164},{"ID":"85","Name":"WEST ELEVATOR ENTRANCE (WEST OF 17th & I STs)","StationCode1":"C03","StationCode2":"","Description":"Entrance elevator is located at the northwest corner of 18th St NW and I St NW.","Lat":38.901502,"Lon":-77.041888},{"ID":"88","Name":"WEST ENTRANCE 2(BUIDLING INTERIOR)","StationCode1":"C03","StationCode2":"","Description":"Station entrance escalator is located in the building (International Square) food court","Lat":38.90149,"Lon":-77.042054},{"ID":"87","Name":"WEST ENTRANCE (WEST OF 17th & I STs)","StationCode1":"C03","StationCode2":"","Description":"Station entrance escalator is located in the building courtyard, at the northwest corner of 18 St NW and I St NW.","Lat":38.901622,"Lon":-77.041965},{"ID":"7","Name":"SOUTH E

In [71]:
import json
import numpy as np
import pandas as pd
from geopy.distance import vincenty

In [55]:
metro1 = pd.read_json('/Users/matthewcassi/Documents/metro.json')

In [56]:
metro1.head()

,Entrances
0,"{'ID': '86', 'Name': 'EAST ENTRANCE (EAST OF 1..."
1,"{'ID': '85', 'Name': 'WEST ELEVATOR ENTRANCE (..."
2,"{'ID': '88', 'Name': 'WEST ENTRANCE 2(BUIDLING..."
3,"{'ID': '87', 'Name': 'WEST ENTRANCE (WEST OF 1..."
4,"{'ID': '7', 'Name': 'SOUTH ENTRANCE (CONNECTIC..."


In [59]:
metro1 = metro1['Entrances'].apply(pd.Series)

In [60]:
metro1.head()

,Description,ID,Lat,Lon,Name,StationCode1,StationCode2
0,Building entrance from the southeast corner of...,86,38.901169,-77.039164,EAST ENTRANCE (EAST OF 17th & I STs),C03,
1,Entrance elevator is located at the northwest ...,85,38.901502,-77.041888,WEST ELEVATOR ENTRANCE (WEST OF 17th & I STs),C03,
2,Station entrance escalator is located in the b...,88,38.901490,-77.042054,WEST ENTRANCE 2(BUIDLING INTERIOR),C03,
3,Station entrance escalator is located in the b...,87,38.901622,-77.041965,WEST ENTRANCE (WEST OF 17th & I STs),C03,
4,Building entrance from the northeast corner of...,7,38.902767,-77.039121,SOUTH ENTRANCE (CONNECTICUT & K ST),A02,


In [61]:
metro1 = metro1.drop(['Description', 'StationCode1', 'StationCode2'], 1)

In [62]:
metro1['lat_lon'] = list(zip(metro1.Lat, metro1.Lon))

In [63]:
metro1.head()

,ID,Lat,Lon,Name,lat_lon
0,86,38.901169,-77.039164,EAST ENTRANCE (EAST OF 17th & I STs),"(38.901169, -77.039164)"
1,85,38.901502,-77.041888,WEST ELEVATOR ENTRANCE (WEST OF 17th & I STs),"(38.901502, -77.041888)"
2,88,38.901490,-77.042054,WEST ENTRANCE 2(BUIDLING INTERIOR),"(38.90149, -77.042054)"
3,87,38.901622,-77.041965,WEST ENTRANCE (WEST OF 17th & I STs),"(38.901622, -77.041965)"
4,7,38.902767,-77.039121,SOUTH ENTRANCE (CONNECTICUT & K ST),"(38.902767, -77.039121)"


In [64]:
metro_stations = list(metro1['lat_lon'])

In [65]:
metro_stations

[(38.901169000000003, -77.039164),
 (38.901502000000001, -77.041888),
 (38.901490000000003, -77.042053999999993),
 (38.901622000000003, -77.041965000000005),
 (38.902766999999997, -77.039120999999994),
 (38.903021000000003, -77.039314000000005),
 (38.901130000000002, -77.034947000000003),
 (38.903525000000002, -77.040186000000006),
 (38.903925000000001, -77.039787000000004),
 (38.900914999999998, -77.032222000000004),
 (38.901097999999998, -77.032302999999999),
 (38.901142, -77.032318000000004),
 (38.900866000000001, -77.050370999999998),
 (38.898026000000002, -77.029379000000006),
 (38.898432999999997, -77.029402000000005),
 (38.900824, -77.050855999999996),
 (38.897146999999997, -77.028566999999995),
 (38.898985000000003, -77.028227999999999),
 (38.893827000000002, -77.028987999999998),
 (38.898497999999996, -77.027902999999995),
 (38.898919999999997, -77.027913999999996),
 (38.893773000000003, -77.028317999999999),
 (38.898072999999997, -77.026788999999994),
 (38.898364999999998, -7

In [66]:
bikeshare = pd.read_csv('/Users/matthewcassi/Documents/Bike-Sharing-Dataset/less_bikeshare_lat_lon.csv', 
                        parse_dates=['Start date', 'End date', 'start_date_short', 'end_date_short'])
# Removing the Unnamed column
bikeshare.drop('Unnamed: 0', 1, inplace=True)

In [67]:
include = ['start_station','start_lat','start_lon']

In [68]:
bikeshare = bikeshare[include]

In [69]:
bikeshare.head()

,start_station,start_lat,start_lon
0,14th & Harvard St NW,38.926800,-77.032200
1,Lincoln Park / 13th & East Capitol St NE,38.890461,-76.988355
2,14th & R St NW,38.912939,-77.032085
3,Park Rd & Holmead Pl NW,38.930800,-77.031500
4,37th & O St NW / Georgetown University,38.907740,-77.071652


In [70]:
bikeshare['lat_lon'] = list(zip(bikeshare.start_lat, bikeshare.start_lon))
bikeshare = bikeshare.drop(['start_lat', 'start_lon'], 1)
bikeshare.head()

,start_station,lat_lon
0,14th & Harvard St NW,"(38.9268, -77.0322)"
1,Lincoln Park / 13th & East Capitol St NE,"(38.890461, -76.988355)"
2,14th & R St NW,"(38.912939, -77.032085)"
3,Park Rd & Holmead Pl NW,"(38.9308, -77.0315)"
4,37th & O St NW / Georgetown University,"(38.90774, -77.071652)"


In [81]:
bikeshare = bikeshare.drop_duplicates()
bikeshare.head()

,start_station,lat_lon
0,14th & Harvard St NW,"(38.9268, -77.0322)"
1,Lincoln Park / 13th & East Capitol St NE,"(38.890461, -76.988355)"
2,14th & R St NW,"(38.912939, -77.032085)"
3,Park Rd & Holmead Pl NW,"(38.9308, -77.0315)"
4,37th & O St NW / Georgetown University,"(38.90774, -77.071652)"


In [82]:
def metro_dist(row):
    distances = []
    for i in metro_stations:
        distance = vincenty(row['lat_lon'], i).miles
        distances.append(distance)
    min_d = min(distances)
    return min_d

In [84]:
bikeshare['metro_dist'] = bike_stations.apply(metro_dist, axis=1)

In [76]:
metro_dist(bike_stations.iloc[1])

0.586599298063801

In [85]:
bikeshare.head()

,start_station,lat_lon,metro_dist
0,14th & Harvard St NW,"(38.9268, -77.0322)",0.125465
1,Lincoln Park / 13th & East Capitol St NE,"(38.890461, -76.988355)",0.586599
2,14th & R St NW,"(38.912939, -77.032085)",0.309956
3,Park Rd & Holmead Pl NW,"(38.9308, -77.0315)",0.135451
4,37th & O St NW / Georgetown University,"(38.90774, -77.071652)",0.810842


In [86]:
bikeshare = bikeshare.drop('lat_lon', 1)

In [87]:
bikeshare.head()

,start_station,metro_dist
0,14th & Harvard St NW,0.125465
1,Lincoln Park / 13th & East Capitol St NE,0.586599
2,14th & R St NW,0.309956
3,Park Rd & Holmead Pl NW,0.135451
4,37th & O St NW / Georgetown University,0.810842


In [99]:
dict_map = {}

In [97]:
def dictionary(row):
    stat = row['start_station']
    distance = row['metro_dist']
    dict_map[stat] = distance

In [100]:
bikeshare.apply(dictionary, axis=1)

0          None
1          None
2          None
3          None
4          None
6          None
7          None
8          None
9          None
10         None
11         None
12         None
13         None
14         None
16         None
19         None
20         None
21         None
22         None
23         None
29         None
30         None
33         None
36         None
37         None
43         None
45         None
47         None
50         None
54         None
           ... 
700857     None
700870     None
700881     None
700883     None
700911     None
700931     None
700959     None
700989     None
701020     None
701026     None
701046     None
701125     None
701259     None
701343     None
701718     None
711749     None
715035     None
939461     None
939699     None
1000794    None
1036311    None
1041609    None
1042224    None
1050027    None
1058553    None
1058739    None
1079995    None
1110594    None
1149487    None
1149912    None
Length: 131, dtype: obje

In [101]:
dict_map

{'10th & Monroe St NE': 0.10384473083526867,
 '10th & U St NW': 0.0337362531647064,
 '10th St & Constitution Ave NW': 0.13446531517503682,
 '11th & H St NE': 0.771532903686702,
 '11th & Kenyon St NW': 0.25156692572275485,
 '12th & Army Navy Dr': 0.33194886724307876,
 '12th & Newton St NE': 0.17949932317781622,
 '13th & D St NE': 0.7226111257935186,
 '13th & H St NE': 0.9106514297916092,
 '13th St & New York Ave NW': 0.12409417540342062,
 '14th & D St NW / Ronald Reagan Building': 0.1494354206099873,
 '14th & D St SE': 0.21040207318145623,
 '14th & G St NW': 0.13177552785719238,
 '14th & Harvard St NW': 0.12546507470734214,
 '14th & R St NW': 0.30995569254958466,
 '14th & Rhode Island Ave NW': 0.5144589328667551,
 '14th & V St NW': 0.16557293412051266,
 '14th St & Spring Rd NW': 0.43577105742914823,
 '14th St Heights / 14th & Crittenden St NW': 0.850354021326083,
 '15th & Crystal Dr': 0.165341465568742,
 '15th & P St NW': 0.48593675973974243,
 '15th St & Massachusetts Ave SE': 0.3560952

In [102]:
bikeshare.head()

,start_station,metro_dist
0,14th & Harvard St NW,0.125465
1,Lincoln Park / 13th & East Capitol St NE,0.586599
2,14th & R St NW,0.309956
3,Park Rd & Holmead Pl NW,0.135451
4,37th & O St NW / Georgetown University,0.810842


In [103]:
bikeshare.to_csv('station_distances.csv')